In [ ]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, mannwhitneyu, wilcoxon, ttest_rel, ttest_ind
import seaborn as sns
from matplotlib.patches import FancyArrowPatch
from matplotlib.colors import ColorConverter

remap_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(remap_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
sys.path.append(lab_path)
os.chdir(lab_path)
sys.path.append(lab_path)
print(lab_path)

In [ ]:
# ANT_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_remapping\ANT"
# NON_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_remapping\NON"
# B6_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_remapping\B6"
# dfs = []
# for path in [ANT_path, NON_path, B6_path]:
#     for animal_dir in os.listdir(path):
#         for csv_file in os.listdir(path + r'\\' + animal_dir):
#             if csv_file.endswith('.xlsx'):
#                 file_df = pd.read_excel(path + r'\\' + animal_dir + r'\\' + csv_file)
#                 # add column for genotype
#                 if path == ANT_path:
#                     file_df['group'] = 'ANT'
#                 elif path == NON_path:
#                     file_df['group'] = 'NON'
#                 elif path == B6_path:
#                     file_df['group'] = 'B6'
#                 dfs.append(file_df)

# df_remapping = pd.concat(dfs, ignore_index=True)
# df_remapping.columns

df_remapping = pd.read_excel(r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_neurofunc_remapping_joined\full_remapping_swap_modified_combined.xlsx")
df_remapping.columns

In [ ]:
# should be empty
df_remapping[df_remapping['cylinder'] == False]['signature'].unique()

In [ ]:
# # Match remapping csvs to neurofunc csvss

# ANT_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_neurofunc\ANT"
# NON_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_neurofunc\NON"
# B6_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_neurofunc\B6"
# dfs = []
# for path in [ANT_path, NON_path, B6_path]:
#     for csv_file in os.listdir(path):
#         if csv_file.endswith('.xlsx') and '~$' not in csv_file:
#             print(csv_file)
#             file_df = pd.read_excel(path + r'\\' + csv_file)
#             name = csv_file.split('_')[0]
#             for i, row in file_df.iterrows():
#                 signature = row['Session']
#                 # print(name)
#                 if name not in signature:
#                     signature = name + '_' + signature
#                     file_df.at[i, 'Session'] = signature

#             # add column for genotype
#             if path == ANT_path:
#                 file_df['group'] = 'ANT'
#             elif path == NON_path:
#                 file_df['group'] = 'NON'
#             elif path == B6_path:
#                 file_df['group'] = 'B6'
#             dfs.append(file_df)

# df_neurofunc = pd.concat(dfs, ignore_index=True)

df_neurofunc = pd.read_excel(r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_neurofunc_remapping_joined\full_neurofunc_swap_modified_combined.xlsx")

from datetime import datetime

NON_set_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_set_files\NON"
ANT_set_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_set_files\ANT"
B6_set_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\LEC_set_files\B6"
# insert column for date_time into df_neurofunc
df_neurofunc['date_time'] = pd.NaT
df_neurofunc['ses_ct'] = pd.NaT

for path in [ANT_set_path, NON_set_path, B6_set_path]:
    for animal_dir in os.listdir(path):
        for set_dir in os.listdir(path + r'\\' + animal_dir):
            date_times = {}
            # sigs = []
            for set_file in os.listdir(path + r'\\' + animal_dir + r'\\' + set_dir):
                # print(set_file)
                assert set_file.endswith('.set')
                ses_signature = set_file.split('.')[0]

                # sigs.append(ses_signature)
                # print(set_file)
                with open(path + r'\\' + animal_dir + r'\\' + set_dir + r'\\' + set_file, 'r') as f:
                    for line in f:
                        if 'trial_time' in str(line):
                            # trial_time = line.decode(encoding='UTF-8').split(" ")[1]
                            trial_time = line.split(" ")[1]
                        if 'trial_date' in str(line):
                            trial_date = line.split(" ")[2:]

                    day, month, year = trial_date
                    month = datetime.strptime(str(month), '%b').month
                    hour, minute, second = trial_time.split(':')
                    date_time = datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))

                # assert ses_signature not in date_times.keys(), 'Session signature {} already in dictionary'.format(ses_signature)
                # assert date_time not in date_times.values(), 'Date time {} already in dictionary'.format(date_time) 
                date_times[ses_signature] = date_time

                print(date_time, ses_signature)

            # sort dictionary by date_time
            sigs = list(date_times.keys())
            date_times = list(date_times.values())
            sorted_sigs = [x for _, x in sorted(zip(date_times, sigs))]
            date_times = sorted(date_times)
            print(date_times, sorted_sigs)
            ses_ct = 1
            for ses in sorted_sigs:


                if len(df_neurofunc[df_neurofunc['Session'] == ses]) == 0 and len(sorted_sigs) > 1:
                    print('Session {} not found in df_neurofunc'.format(ses))
                # else:
                    # add date time column
                else:
                    if 'ANT-135a-7_20180604' in ses:
                        print('gotohere')
                        print(ses_ct - 1)
                        df_neurofunc.loc[(df_neurofunc['Session'] == ses) & (df_neurofunc['Tetrode'] != 7), 'date_time'] = date_times[ses_ct - 1]
                        df_neurofunc.loc[(df_neurofunc['Session'] == ses) & (df_neurofunc['Tetrode'] != 7), 'ses_ct'] = ses_ct 
                        df_neurofunc.loc[(df_neurofunc['Session'] == ses) & (df_neurofunc['Tetrode'] == 7), 'date_time'] = date_times[ses_ct - 1]
                        df_neurofunc.loc[(df_neurofunc['Session'] == ses) & (df_neurofunc['Tetrode'] == 7), 'ses_ct'] = ses_ct - 1
                        ses_ct += 1
        
                    else:
                        df_neurofunc.loc[df_neurofunc['Session'] == ses, 'date_time'] = date_times[ses_ct - 1]
                        df_neurofunc.loc[df_neurofunc['Session'] == ses, 'ses_ct'] = ses_ct
                        ses_ct += 1


                    
# cast ses_ct as int
df_neurofunc['ses_ct'] = df_neurofunc['ses_ct'].astype(int)
print(df_neurofunc.columns)

In [ ]:
# should be 1 to 7
assert len(df_neurofunc['ses_ct'].unique()) == 7

In [ ]:
# should be empty
assert len(df_neurofunc[df_neurofunc['date_time'].isna()]['Session'].unique()) == 0

In [ ]:
# print Session where date_time is NaT
print(df_neurofunc[df_neurofunc['date_time'].isna()]['Session'].unique())
print(len(df_neurofunc[df_neurofunc['date_time'].isna()]['Session'].unique()), len(df_neurofunc['Session'].unique()))

In [ ]:
# Need to match neurofunc and remapping
import re
from scripts.batch_map.LEC_naming import LEC_naming_format, extract_name_lec

def _check_single_format(filename, fformat, fxn):
    if re.match(str(fformat), str(filename)) is not None:
        return list(fxn(filename))

# For neurofunc need to add extract, date, depth, name, stim 
# iterate thru rows of df_neurofunc and extract from signature
for i, row in df_neurofunc.iterrows():
    # extract
    fname = row['Session']
    
    if 'CAGE' in fname:
        # drop row
        df_neurofunc.drop(i, inplace=True)
    else:
        print(fname)
        group, name = extract_name_lec(fname)
        formats = LEC_naming_format[group][name]['object']

        for fformat in list(formats.keys()):
            checked = _check_single_format(fname, fformat, formats[fformat])
            if checked is not None:
                break

        stim, depth, name, date = checked
        
        df_neurofunc.at[i, 'date'] = date
        df_neurofunc.at[i, 'depth'] = str(int(depth))
        df_neurofunc.at[i, 'name'] = name
        df_neurofunc.at[i, 'stim'] = stim

In [ ]:
# neurofunc_row_identifiers = ['name','Tetrode', 'Cell ID', 'date', 'depth', 'stim']
# remapping_row_identifiers = ['name','tetrode', 'unit_id', 'date', 'depth','object_location']

# df_remapping_to_merge = df_remapping[df_remapping['score'] == 'whole']

# for row in neurofunc_row_identifiers:
#     df_neurofunc[row] = df_neurofunc[row].astype(str)
# for row in remapping_row_identifiers:
#     df_remapping_to_merge[row] = df_remapping_to_merge[row].astype(str)

# matched_df = None
# for i, row in df_remapping_to_merge.iterrows():
#     # find row in df_neurofunc that mathces using identifiers
#     for j, row2 in df_neurofunc.iterrows():
#         matched = True
#         for k in range(len(neurofunc_row_identifiers)):
#             print('pair ' + str(neurofunc_row_identifiers[k]))
#             print('new2')
#             print(row2[neurofunc_row_identifiers[k]])
#             print('new')
#             print(row[remapping_row_identifiers[k]])
            
#             if row2[neurofunc_row_identifiers[k]] != row[remapping_row_identifiers[k]]:
#                 matched = False

#         # if row2[neurofunc_row_identifiers] == row[remapping_row_identifiers]:
#         if matched == True:
#             if matched_df is None:
#                 row = pd.concat([row, row2], axis=0)
#                 matched_df = row
#             else:
#                 row = pd.concat([row, row2], axis=0)
#                 matched_df = pd.concat([matched_df, row], axis=1)
        



In [ ]:
import pandas as pd

# Define the common columns
neurofunc_row_identifiers = ['name', 'Tetrode', 'Cell ID', 'date', 'depth', 'stim']
remapping_row_identifiers = ['name', 'tetrode', 'unit_id', 'date', 'depth', 'object_location']

for row in neurofunc_row_identifiers:
    df_neurofunc[row] = df_neurofunc[row].astype(str)
for row in remapping_row_identifiers:
    df_remapping[row] = df_remapping[row].astype(str)
df_neurofunc['Session'] = df_neurofunc['Session'].astype(str)
df_remapping['signature'] = df_remapping['signature'].astype(str)


In [ ]:
swap_df_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\Sifting_Through_Cells_Modified.xlsx"
swap_df_ANT = pd.read_excel(swap_df_path, sheet_name='ANT')
swap_df_B6 = pd.read_excel(swap_df_path, sheet_name='B6')
swap_df_NON = pd.read_excel(swap_df_path, sheet_name='NON')
swap_df = pd.concat([swap_df_ANT, swap_df_B6, swap_df_NON])
swap_df['CATEGORY'] = swap_df['CATEGORY'].astype(str)
swap_df['CATEGORY'] = swap_df['CATEGORY'].str.replace('.png', '')

In [ ]:

# Create an empty list to store the merged rows
merged_rows = []

# Iterate through each row in df_remapping
counter = 0
remapping_rows = []
for remapping_row in df_remapping.itertuples(index=False):
    # Extract the identifiers from the remapping row
    remapping_identifiers = [getattr(remapping_row, col) for col in remapping_row_identifiers]

    # Find the matching row in df_neurofunc based on the identifiers
    matching_row = df_neurofunc.loc[
        (df_neurofunc[neurofunc_row_identifiers] == remapping_identifiers).all(axis=1)
    ]

    if len(matching_row) > 1:


        """ If unnamed row in csv then change to [7] instead of [6]. Or double check what row has seession_1, session_2, etc..."""
        rw = remapping_row[6]
        assert 'session' in rw.lower(), 'Remapping row is {}'.format(rw)
        remapping_row_ses_ct = int(rw.split('_')[-1])

        pre_matched = matching_row.copy()
        matching_row = matching_row[matching_row['ses_ct'] == remapping_row_ses_ct]
        if len(matching_row) == 0:
            print('no match found')
            print(remapping_row)
            print(pre_matched)
            print(matching_row)

    # Check if a matching row was found
    if len(matching_row) == 0:
        print("Found {} matching rows for {}".format(len(matching_row), remapping_identifiers))
        print(matching_row['Session'], getattr(remapping_row, 'signature'))
    elif len(matching_row) > 1:
        # check all rows are matching on spatial info score 
        prev = None
        for i, row in matching_row.iterrows():
            if prev is None:
                prev = row['spike_count']
            else:
                if prev != row['spike_count']:
                    print('multiple matches')
                    print(matching_row)
        

    aid = remapping_row.name
    date = remapping_row.date
    tet = remapping_row.tetrode
    cell_label = remapping_row.unit_id
    if '-' in str(date):
        date = date.split('-')[0]

    swap_df_row_id = aid + '_' + date + '_' + tet + '_' + str(cell_label)
    swap_df_row_ids = np.where(swap_df['CATEGORY'].to_numpy() == swap_df_row_id)[0]

    assert len(swap_df_row_ids) <= 1, 'Multiple rows found for {}'.format(swap_df_row_id)

    if len(swap_df_row_ids) != 0:
        df_sample = swap_df.iloc[swap_df_row_ids[0]:swap_df_row_ids[0]+1]
        row = df_sample['SWAP/DROP/KEEP']
        row = row.values[0]

        if row != row:
            print('row is nan for {}'.format(swap_df_row_id))

        # if row != row or 'keep' not in row.lower():
        if row != row or 'drop' in row.lower():
            pass
        else:
            assert len(matching_row) == 1, 'Multiple rows found for {}'.format(matching_row)
            merged_rows.append(matching_row)
            remapping_rows.append(remapping_row)
            counter += 1

            if len(merged_rows) != counter:
                print('error {} != {}'.format(len(merged_rows), counter))
                print(matching_row)


merged_df = pd.concat(merged_rows, ignore_index=True)
remapping_merged_df = pd.DataFrame(remapping_rows, columns=df_remapping.columns)


In [ ]:
lns = []
firsts = []
seses = []
for row in merged_rows:
    ln = len(row)
    first = row.iloc[0]
    lns.append(ln)
    if ln == 2:
        sese = row['Session'].unique()
        seses.append(sese)
print(np.unique(seses))
# should be empty

In [ ]:
df_full = pd.concat([remapping_merged_df, merged_df], axis=1)
print(df_full.shape)
df_full.to_excel('df_full_LEC_keep_swapped2.xlsx')

In [ ]:
remapping_merged_df.shape

In [ ]:
merged_df.shape

In [ ]:
print(df_neurofunc.shape, df_remapping.shape, merged_df.shape)

In [ ]:
# remapping_identifiers = [getattr(remapping_row, col) for col in remapping_row_identifiers]

#     # Find the matching row in df_neurofunc based on the identifiers
# matching_row = df_neurofunc.loc[
#         (df_neurofunc[neurofunc_row_identifiers] == remapping_identifiers).all(axis=1)
#     ]

In [ ]:
print(len(df_neurofunc.groupby(['Session']).groups.keys()))
print(len(df_remapping.groupby(['signature'])))
for ses in df_neurofunc.groupby(['Session']).groups.keys():
    if ses not in df_remapping.groupby(['signature']).groups.keys():
        print(ses)

In [ ]:
spl = df_remapping[(df_remapping['signature'] == 'ANT-135a-7_20180604-ROUND-3225-180-2') & (df_remapping['tetrode'] == '7')]
for remapping_row in spl.itertuples(index=False):
    # Extract the identifiers from the remapping row
    remapping_identifiers = [getattr(remapping_row, col) for col in remapping_row_identifiers]
    print(remapping_row)
    # Find the matching row in df_neurofunc based on the identifiers
    matching_row = df_neurofunc.loc[
        (df_neurofunc[neurofunc_row_identifiers] == remapping_identifiers).all(axis=1)
    ]
    print(matching_row['ses_ct'])


In [ ]:
# concat df remapping and df merged 
# df_full = pd.concat([df_remapping, merged_df], axis=1)
# df_full.to_excel('df_full_LEC_only_keep.xlsx')

In [ ]:
df_full = pd.concat([df_remapping, merged_df], axis=1)
# drop duplicate columns
df_full = df_full.loc[:,~df_full.columns.duplicated()]

dtemp = df_full[df_full['score'] == 'whole']
lst = dtemp.groupby(['name', 'depth', 'date','tetrode', 'unit_id','session_id','object_location'])
def _single(dtemp, grp):
    if len(dtemp.groupby(['name', 'depth', 'date','tetrode', 'unit_id','session_id','object_location']).groups[grp]) != 1:
        print(grp)
        print(dtemp.groupby(['name', 'depth', 'date','tetrode', 'unit_id','session_id','object_location']).groups[grp])
        # stop()

list(map(lambda x: _single(dtemp, x), lst.groups))

# fld_counts['field_count'].unique()

In [ ]:
for row in merged_rows:
    ln = len(row)
    if ln != 1:
        print(row['Session'])